# AQR Multi-Factor ETF Strategy - Portfolio Monitoring Dashboard

**Purpose**: Monitor portfolio performance, positions, and factor exposures

**Data Source**: `results/portfolio_tracking.xlsx`

**Last Updated**: 2025-10-17

---

## Overview

This notebook provides comprehensive monitoring of the live portfolio:
1. **Performance Analysis**: Returns, drawdowns, Sharpe ratio vs benchmarks
2. **Position Analysis**: Current allocations, concentration, factor exposures
3. **Trade History**: Rebalancing patterns, turnover, execution tracking
4. **Risk Metrics**: Volatility, VaR, stop-loss distances
5. **Factor Attribution**: Contribution of each factor to performance

In [ ]:
import sys
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from datetime import datetime, timedelta

# Plotting setup
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

# Project paths
PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))

TRACKING_FILE = PROJECT_ROOT / 'results' / 'portfolio_tracking.xlsx'

print(f"Project root: {PROJECT_ROOT}")
print(f"Tracking file: {TRACKING_FILE}")
print(f"Tracking file exists: {TRACKING_FILE.exists()}")

## 1. Load Portfolio Data

In [ ]:
def load_portfolio_data():
    """Load all sheets from tracking workbook"""
    if not TRACKING_FILE.exists():
        print(f"ERROR: Tracking file not found: {TRACKING_FILE}")
        print("Please run: python scripts/run_weekly_portfolio.py")
        return None, None, None, None
    
    try:
        positions = pd.read_excel(TRACKING_FILE, sheet_name='Positions')
        trades = pd.read_excel(TRACKING_FILE, sheet_name='Trades')
        performance = pd.read_excel(TRACKING_FILE, sheet_name='Performance')
        metadata = pd.read_excel(TRACKING_FILE, sheet_name='Metadata')
        
        # Convert timestamps
        for df in [positions, trades, performance, metadata]:
            if 'timestamp' in df.columns:
                df['timestamp'] = pd.to_datetime(df['timestamp'])
        
        print(f"Loaded {len(positions)} position records")
        print(f"Loaded {len(trades)} trade records")
        print(f"Loaded {len(performance)} performance records")
        print(f"Loaded {len(metadata)} metadata records")
        
        return positions, trades, performance, metadata
    
    except Exception as e:
        print(f"Error loading data: {e}")
        return None, None, None, None

# Load data
positions_df, trades_df, performance_df, metadata_df = load_portfolio_data()

In [ ]:
# Display data summary
if positions_df is not None:
    print("\n=== Data Summary ===")
    print(f"Date range: {positions_df['timestamp'].min()} to {positions_df['timestamp'].max()}")
    print(f"Number of rebalances: {len(positions_df['timestamp'].unique())}")
    print(f"Total trades: {len(trades_df) if trades_df is not None else 0}")
    
    # Current portfolio
    latest_timestamp = positions_df['timestamp'].max()
    current_positions = positions_df[positions_df['timestamp'] == latest_timestamp]
    print(f"\nCurrent portfolio ({latest_timestamp.date()}):")
    print(f"  Positions: {len(current_positions)}")
    print(f"  Total value: ${current_positions['value'].sum():,.2f}")
    print(f"  Largest position: {current_positions.loc[current_positions['value'].idxmax(), 'ticker']} "
          f"(${current_positions['value'].max():,.2f}, {current_positions['weight'].max():.1%})")

## 2. Current Portfolio Analysis

In [ ]:
if positions_df is not None and len(positions_df) > 0:
    # Get most recent positions
    latest_timestamp = positions_df['timestamp'].max()
    current = positions_df[positions_df['timestamp'] == latest_timestamp].copy()
    current = current.sort_values('value', ascending=False)
    
    print(f"\n=== Current Portfolio as of {latest_timestamp.date()} ===")
    print(current[['ticker', 'shares', 'price', 'value', 'weight', 'factor_score']].to_string(index=False))
    
    # Summary statistics
    print(f"\n=== Portfolio Statistics ===")
    print(f"Number of positions: {len(current)}")
    print(f"Total value: ${current['value'].sum():,.2f}")
    print(f"Average position size: ${current['value'].mean():,.2f} ({current['weight'].mean():.2%})")
    print(f"Largest position: {current['weight'].max():.2%}")
    print(f"Smallest position: {current['weight'].min():.2%}")
    print(f"Average factor score: {current['factor_score'].mean():.3f}")
    print(f"Factor score range: {current['factor_score'].min():.3f} to {current['factor_score'].max():.3f}")

In [ ]:
# Visualize current allocations
if positions_df is not None and len(positions_df) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Pie chart of top 10 positions
    top10 = current.nlargest(10, 'value')
    other_value = current[~current.index.isin(top10.index)]['value'].sum()
    
    pie_data = pd.concat([
        pd.Series(top10['value'].values, index=top10['ticker'].values),
        pd.Series([other_value], index=['Other'])
    ])
    
    axes[0].pie(pie_data, labels=pie_data.index, autopct='%1.1f%%', startangle=90)
    axes[0].set_title('Portfolio Allocation (Top 10 + Other)', fontsize=14, fontweight='bold')
    
    # Bar chart of all positions
    axes[1].barh(current['ticker'], current['weight'] * 100)
    axes[1].set_xlabel('Weight (%)', fontsize=12)
    axes[1].set_ylabel('Ticker', fontsize=12)
    axes[1].set_title('All Position Weights', fontsize=14, fontweight='bold')
    axes[1].grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Factor score distribution
if positions_df is not None and len(positions_df) > 0:
    fig, ax = plt.subplots(figsize=(10, 6))
    
    ax.bar(current['ticker'], current['factor_score'], color='steelblue')
    ax.axhline(y=current['factor_score'].mean(), color='red', linestyle='--', 
               label=f'Average: {current["factor_score"].mean():.3f}')
    ax.set_xlabel('Ticker', fontsize=12)
    ax.set_ylabel('Factor Score', fontsize=12)
    ax.set_title('Factor Scores of Current Holdings', fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(axis='y', alpha=0.3)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

## 3. Performance Analysis

In [ ]:
def calculate_realized_returns(positions_df):
    """Calculate actual returns from position history"""
    if positions_df is None or len(positions_df) == 0:
        return None
    
    # Get portfolio values at each timestamp
    portfolio_values = positions_df.groupby('timestamp')['value'].sum().sort_index()
    
    # Calculate returns
    returns = portfolio_values.pct_change().dropna()
    
    # Calculate cumulative returns
    cum_returns = (1 + returns).cumprod() - 1
    
    return pd.DataFrame({
        'portfolio_value': portfolio_values,
        'returns': returns,
        'cum_returns': cum_returns
    })

# Calculate returns
returns_df = calculate_realized_returns(positions_df)

if returns_df is not None and len(returns_df) > 1:
    print("\n=== Performance Metrics ===")
    total_return = returns_df['cum_returns'].iloc[-1]
    num_periods = len(returns_df)
    
    print(f"Total return: {total_return:.2%}")
    print(f"Number of rebalances: {num_periods}")
    print(f"Average return per period: {returns_df['returns'].mean():.2%}")
    print(f"Volatility per period: {returns_df['returns'].std():.2%}")
    
    if returns_df['returns'].std() > 0:
        sharpe = returns_df['returns'].mean() / returns_df['returns'].std()
        print(f"Sharpe ratio: {sharpe:.2f}")
    
    # Drawdown
    cum_returns_series = returns_df['cum_returns'] + 1
    running_max = cum_returns_series.expanding().max()
    drawdown = (cum_returns_series - running_max) / running_max
    max_dd = drawdown.min()
    print(f"Maximum drawdown: {max_dd:.2%}")
else:
    print("\nNot enough data to calculate returns (need at least 2 rebalances)")

In [ ]:
# Download benchmark (SPY) for comparison
def get_benchmark_returns(start_date, end_date):
    """Download SPY returns for comparison"""
    try:
        spy = yf.Ticker('SPY')
        hist = spy.history(start=start_date, end=end_date)
        returns = hist['Close'].pct_change().dropna()
        return returns
    except Exception as e:
        print(f"Could not download SPY: {e}")
        return None

if returns_df is not None and len(returns_df) > 0:
    start_date = returns_df.index.min()
    end_date = returns_df.index.max()
    
    spy_returns = get_benchmark_returns(start_date, end_date)
    
    if spy_returns is not None:
        # Resample SPY to match portfolio dates
        spy_aligned = spy_returns.reindex(returns_df.index, method='nearest')
        spy_cum = (1 + spy_aligned).cumprod() - 1
        
        print(f"\n=== Benchmark Comparison (SPY) ===")
        print(f"Portfolio return: {returns_df['cum_returns'].iloc[-1]:.2%}")
        print(f"SPY return: {spy_cum.iloc[-1]:.2%}")
        print(f"Excess return: {(returns_df['cum_returns'].iloc[-1] - spy_cum.iloc[-1]):.2%}")

In [ ]:
# Plot performance
if returns_df is not None and len(returns_df) > 1:
    fig, axes = plt.subplots(2, 1, figsize=(14, 10))
    
    # Cumulative returns
    axes[0].plot(returns_df.index, returns_df['cum_returns'] * 100, 
                 marker='o', linewidth=2, label='Portfolio', color='steelblue')
    
    if spy_returns is not None:
        axes[0].plot(returns_df.index, spy_cum * 100, 
                     marker='s', linewidth=2, label='SPY', color='orange', alpha=0.7)
    
    axes[0].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    axes[0].set_xlabel('Date', fontsize=12)
    axes[0].set_ylabel('Cumulative Return (%)', fontsize=12)
    axes[0].set_title('Cumulative Returns', fontsize=14, fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Period returns
    axes[1].bar(returns_df.index, returns_df['returns'] * 100, color='steelblue', alpha=0.7)
    axes[1].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    axes[1].set_xlabel('Date', fontsize=12)
    axes[1].set_ylabel('Period Return (%)', fontsize=12)
    axes[1].set_title('Period Returns', fontsize=14, fontweight='bold')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Drawdown chart
if returns_df is not None and len(returns_df) > 1:
    fig, ax = plt.subplots(figsize=(14, 6))
    
    cum_returns_series = returns_df['cum_returns'] + 1
    running_max = cum_returns_series.expanding().max()
    drawdown = (cum_returns_series - running_max) / running_max
    
    ax.fill_between(drawdown.index, drawdown * 100, 0, color='red', alpha=0.3)
    ax.plot(drawdown.index, drawdown * 100, color='darkred', linewidth=2)
    ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax.set_xlabel('Date', fontsize=12)
    ax.set_ylabel('Drawdown (%)', fontsize=12)
    ax.set_title('Portfolio Drawdown', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 4. Trade History Analysis

In [ ]:
if trades_df is not None and len(trades_df) > 0:
    print("\n=== Trade History ===")
    print(f"Total trades: {len(trades_df)}")
    print(f"Buy trades: {(trades_df['action'] == 'BUY').sum()}")
    print(f"Sell trades: {(trades_df['action'] == 'SELL').sum()}")
    print(f"Total traded value: ${trades_df['value'].sum():,.2f}")
    
    # Group by rebalance date
    trades_by_date = trades_df.groupby('timestamp').agg({
        'ticker': 'count',
        'value': 'sum'
    }).rename(columns={'ticker': 'num_trades', 'value': 'total_value'})
    
    print(f"\n=== Trades by Rebalance Date ===")
    print(trades_by_date.to_string())
    
    # Most traded tickers
    most_traded = trades_df['ticker'].value_counts().head(10)
    print(f"\n=== Most Frequently Traded Tickers ===")
    print(most_traded.to_string())
else:
    print("\nNo trade history available yet")

In [ ]:
# Visualize trade activity
if trades_df is not None and len(trades_df) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Trades over time
    trades_by_date = trades_df.groupby(['timestamp', 'action']).size().unstack(fill_value=0)
    trades_by_date.plot(kind='bar', stacked=True, ax=axes[0], color=['green', 'red'])
    axes[0].set_xlabel('Date', fontsize=12)
    axes[0].set_ylabel('Number of Trades', fontsize=12)
    axes[0].set_title('Trade Activity by Rebalance', fontsize=14, fontweight='bold')
    axes[0].legend(['BUY', 'SELL'])
    axes[0].grid(axis='y', alpha=0.3)
    plt.setp(axes[0].xaxis.get_majorticklabels(), rotation=45, ha='right')
    
    # Trade value distribution
    axes[1].hist(trades_df['value'], bins=30, color='steelblue', edgecolor='black', alpha=0.7)
    axes[1].set_xlabel('Trade Value ($)', fontsize=12)
    axes[1].set_ylabel('Frequency', fontsize=12)
    axes[1].set_title('Distribution of Trade Sizes', fontsize=14, fontweight='bold')
    axes[1].grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 5. Risk Metrics

In [ ]:
# Get current VIX level
def get_current_vix():
    """Get current VIX level for risk assessment"""
    try:
        vix = yf.Ticker('^VIX')
        hist = vix.history(period='5d')
        latest = hist['Close'].iloc[-1]
        return latest
    except Exception as e:
        print(f"Could not download VIX: {e}")
        return None

vix_level = get_current_vix()

if vix_level is not None:
    print(f"\n=== Current Market Risk ===")
    print(f"VIX Level: {vix_level:.1f}")
    
    if vix_level < 15:
        regime = "Low Volatility"
        stop_loss = 0.15
    elif vix_level <= 25:
        regime = "Normal Volatility"
        stop_loss = 0.12
    else:
        regime = "High Volatility"
        stop_loss = 0.10
    
    print(f"Market Regime: {regime}")
    print(f"Recommended Stop-Loss: {stop_loss:.0%}")

In [ ]:
# Portfolio concentration risk
if positions_df is not None and len(positions_df) > 0:
    latest_timestamp = positions_df['timestamp'].max()
    current = positions_df[positions_df['timestamp'] == latest_timestamp].copy()
    
    print(f"\n=== Concentration Risk ===")
    print(f"Number of positions: {len(current)}")
    print(f"Top position weight: {current['weight'].max():.2%}")
    print(f"Top 3 positions weight: {current.nlargest(3, 'weight')['weight'].sum():.2%}")
    print(f"Top 5 positions weight: {current.nlargest(5, 'weight')['weight'].sum():.2%}")
    print(f"Top 10 positions weight: {current.nlargest(10, 'weight')['weight'].sum():.2%}")
    
    # Herfindahl-Hirschman Index (concentration measure)
    hhi = (current['weight'] ** 2).sum()
    print(f"\nHerfindahl-Hirschman Index: {hhi:.4f}")
    print(f"  (Lower is better; equal-weighted {len(current)} positions = {1/len(current):.4f})")

## 6. Expected vs Realized Performance

In [ ]:
if performance_df is not None and len(performance_df) > 0:
    print("\n=== Expected vs Realized Performance ===")
    print(performance_df[['timestamp', 'expected_return', 'expected_volatility', 'expected_sharpe']].to_string(index=False))
    
    # Compare to realized
    if returns_df is not None and len(returns_df) > 1:
        realized_return = returns_df['returns'].mean()
        realized_vol = returns_df['returns'].std()
        realized_sharpe = realized_return / realized_vol if realized_vol > 0 else 0
        
        expected_return = performance_df['expected_return'].mean()
        expected_vol = performance_df['expected_volatility'].mean()
        expected_sharpe = performance_df['expected_sharpe'].mean()
        
        print(f"\n=== Average Expected ===")
        print(f"Return: {expected_return:.2%}")
        print(f"Volatility: {expected_vol:.2%}")
        print(f"Sharpe: {expected_sharpe:.2f}")
        
        print(f"\n=== Realized (Per Period) ===")
        print(f"Return: {realized_return:.2%}")
        print(f"Volatility: {realized_vol:.2%}")
        print(f"Sharpe: {realized_sharpe:.2f}")

## 7. Execution Summary

In [ ]:
# Check which trades have been executed
if trades_df is not None and len(trades_df) > 0:
    print("\n=== Trade Execution Status ===")
    
    if 'executed' in trades_df.columns:
        executed = trades_df['executed'].sum()
        pending = len(trades_df) - executed
        
        print(f"Executed trades: {executed}")
        print(f"Pending trades: {pending}")
        
        if pending > 0:
            print(f"\n=== Pending Trades to Execute ===")
            pending_trades = trades_df[trades_df['executed'] == False]
            print(pending_trades[['timestamp', 'ticker', 'action', 'shares', 'price', 'value']].to_string(index=False))
            print(f"\nTotal pending value: ${pending_trades['value'].sum():,.2f}")
    else:
        print("No execution status tracked yet")
        print("You can manually update the 'executed' column in the Excel file after executing trades")

## 8. Export Summary Report

In [ ]:
# Create summary report
def create_summary_report():
    """Create a summary report of current portfolio status"""
    report = []
    report.append("=" * 80)
    report.append("AQR MULTI-FACTOR ETF STRATEGY - PORTFOLIO SUMMARY")
    report.append(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    report.append("=" * 80)
    report.append("")
    
    if positions_df is not None and len(positions_df) > 0:
        latest_timestamp = positions_df['timestamp'].max()
        current = positions_df[positions_df['timestamp'] == latest_timestamp]
        
        report.append(f"CURRENT PORTFOLIO (as of {latest_timestamp.date()})")
        report.append(f"  Number of positions: {len(current)}")
        report.append(f"  Total value: ${current['value'].sum():,.2f}")
        report.append(f"  Average factor score: {current['factor_score'].mean():.3f}")
        report.append("")
        
    if returns_df is not None and len(returns_df) > 1:
        report.append("PERFORMANCE")
        report.append(f"  Total return: {returns_df['cum_returns'].iloc[-1]:.2%}")
        report.append(f"  Average period return: {returns_df['returns'].mean():.2%}")
        report.append(f"  Volatility: {returns_df['returns'].std():.2%}")
        
        cum_returns_series = returns_df['cum_returns'] + 1
        running_max = cum_returns_series.expanding().max()
        drawdown = (cum_returns_series - running_max) / running_max
        report.append(f"  Max drawdown: {drawdown.min():.2%}")
        report.append("")
    
    if trades_df is not None and len(trades_df) > 0:
        report.append("TRADE ACTIVITY")
        report.append(f"  Total trades: {len(trades_df)}")
        report.append(f"  Buy trades: {(trades_df['action'] == 'BUY').sum()}")
        report.append(f"  Sell trades: {(trades_df['action'] == 'SELL').sum()}")
        report.append("")
    
    if vix_level is not None:
        report.append("MARKET RISK")
        report.append(f"  VIX Level: {vix_level:.1f}")
        report.append(f"  Market Regime: {regime}")
        report.append(f"  Recommended Stop-Loss: {stop_loss:.0%}")
        report.append("")
    
    report.append("=" * 80)
    
    return "\n".join(report)

summary = create_summary_report()
print(summary)

# Save to file
summary_file = PROJECT_ROOT / 'results' / f'portfolio_summary_{datetime.now().strftime("%Y%m%d_%H%M%S")}.txt'
with open(summary_file, 'w') as f:
    f.write(summary)

print(f"\nSummary saved to: {summary_file}")

## Summary

This notebook provides comprehensive monitoring of your portfolio:

1. **Current Positions**: View all holdings, weights, and factor scores
2. **Performance**: Track returns, drawdowns, and compare to benchmarks
3. **Trade History**: Review all trades and rebalancing activity
4. **Risk Metrics**: Monitor concentration, VIX levels, and stop-loss distances
5. **Execution Tracking**: See which trades are pending execution

**Next Steps**:
- Run `python scripts/run_weekly_portfolio.py` weekly to update the tracking workbook
- Re-run this notebook after each update to see latest results
- Mark trades as executed in the Excel file's Trades sheet
- Review performance trends and adjust parameters if needed

**Files**:
- Tracking workbook: `results/portfolio_tracking.xlsx`
- Summary reports: `results/portfolio_summary_*.txt`
- Log files: `logs/weekly_automation_*.log`